Machine Learning Online Class - Exercise 2: Logistic Regression

# Programming Exercise 2: Logistic Regression
This document is a modification of the Andrew Ng Coursera material for implementation of the problem set in the Julia language. This is NOT part of the official Coursera material and you cannot submit any of those solutions to Coursera or get credit for.

# 0) Introduction

In this exercise, you will implement logistic regression and apply it to two different datasets. Before starting on the programming exercise, we strongly recommend watching the video lectures and completing the review questions for the associated topics.
#### Files included in this exercise

src/ex2.ipynb Jupyter notebook that you will need to modify during the exercise

src/ex2_reg.ipynb Jupyter notebook for the second part of the exercise

data/ex2data1.txt Training set for the first half of the exercise

data/ex2data2.txt Training set for the second half of the exercise

# 1) Logistic Regression

In this part of the exercise, you will build a logistic regression model to predict whether a student gets admitted into a university. Suppose that you are the administrator of a university department and you want to determine each applicant’s chance of admission based on their results on two exams. You have historical data from previous applicants that you can use as a training set for logistic regression. For each training example, you have the applicant’s scores on two exams and the admissions decision. Your task is to build a classification model that estimates an applicant’s probability of admission based the scores from those two exams. This outline and the framework code in ex2.ipynb will guide you through the exercise.

## 1.1) Loading and Visualizing the Data

Before starting to implement any learning algorithm, it is always good to visualize the data if possible. In the first part you will wirte code to load the data and display it on a 2-dimensional plot by calling the function plot_data(). You will now complete the code in plot_data() so that it displays a figure like Figure 1 The two axes are the exam scores, and the positive and negative examples are shown with different markers.

**Instructions**

This file contains code that helps you get started on the logistic
regression exercise. You will need to complete the following functions
in this exericse:

     plot_data()
     sigmoid()
     cost_function()
     predict()
     cost_function_reg()
     
Load the data from the file **../data/ex2data.txt**
The first two columns contain the exam scores and the third column the labels. Your task is to load the data from this file into the two variables **X_data** and **y_data**.

We will use the function *readdlm()* to load the data set into variable **data** now select
the corresponding columns of this data set and assign it to **X_data** and **y_data**

Hints:

You can access a full column of a matrix by using the colon (:) symbol. E.g. **data[:,1]** selects the first column, **data[1,:]** the first row.

The output of **size(X)** should be (100,2) and **size(y)** equals to (100,)

In [1]:
using Plots
using Optim
using BenchmarkTools

filename = "../data/ex2data1.txt"
delimiter = ','
data = readdlm(filename, delimiter);

X_data = data[:, [1, 2]]
y_data = data[:, 3];


@show(size(X_data))
@show(size(y_data));

size(X_data) = (100, 2)
size(y_data) = (100,)


## 1.2) Plotting the Data
We start the exercise by first plotting the data to understand the problem we are working with. Modify *plot_data()* such that you get a figure similar to the one below.

**Find documentation under**
https://juliaplots.github.io/

![title](Fig_1.png)

In [2]:
function plot_data(X, y)
    # In the current form the figure will show yellow squares
    # for all data points. Modify the arguments of 'm', 'c',
    # and 'label' such that you get a nice visual representation
    # such as in Figure 1 from ex2.pdf
    

    admitted = y .== 1.0
    not_admitted = y .== 0.0
    scatter(X_data[admitted, 1], X_data[admitted, 2], m=(0.5, :circle, 4),
            c=:black, label="admitted")
    
    scatter!(X_data[not_admitted, 1], X_data[not_admitted, 2], m=(0.9, :square, 3),
             c=:yellow, label="not admitted")
    xlabel!("score test 1")
    ylabel!("score test 2")
end

plot_data(X_data, y_data)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 50 
 
 
 100 
 
 
 score test 1 
 
 
 score test 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 admitted 
 
 
 
 
 not admitted

## 1.2) Implementation
### 1.2.1) Warmup exercise: sigmoid function
Before you start with the actual cost function, recall that the logistic regression hypothesis is defined as:

$$h_θ(x) = g(θ^Tx).$$

where function $g$ is the sigmoid function. The sigmoid function is defined as:


$$g(z) = \frac{1}{1+e^{−z}}$$


Your first step is to implement this function as *sigmoid()* so it can be
called by the rest of your program. When you are finished, try testing a few
values by calling *sigmoid(x)* with different values of x. For
large positive values of x, the sigmoid should be close to 1, while for large
negative values, the sigmoid should be close to 0. Evaluating *sigmoid(0)*
should give you exactly 0.5. **Your code should also work with vectors and
matrices. For a matrix, your function should perform the sigmoid
function on every element**.

**Hint a)**: https://docs.julialang.org/en/stable/manual/functions/#man-vectorized-1

**Hint b)**: In Julia you can use a more compact syntax to define simple functions than
what we saw for *plot_data()*. Try defining the sigmoid function according to
the example below (but with the actual formula of the sigmoid.):

*my_sinc(z) = sin(z) / z*

In [3]:
sigmoid(z) = 1 ./ (1 .+ exp.(-z))


@show(sigmoid(-10))
@show(sigmoid(0))
@show(sigmoid(100))
@show(sigmoid(-3:3));

sigmoid(-10) = 4.5397868702434395e-5
sigmoid(0) = 0.5
sigmoid(100) = 1.0
sigmoid(-3:3) = [0.0474259, 0.119203, 0.268941, 0.5, 0.731059, 0.880797, 0.952574]


### 1.2.2 Cost Function and Gradient
In this part of the exercise, you will implement the cost and gradient for logistic regression. You neeed to complete the code in **sigmoid()** and **cost_function()**
Recall that the cost function in logistic regression is

$$\frac{1}{m}\sum_{i=1}^m[-y^{(i)}\log(h_\theta(x^{(i)})) - (1-y^{(i)}) \log(1-h_\theta(x^{(i)})) ]$$


and the gradient of the cost is a vector of the same length as θ where the $j^{th}$
element (for j = 0, 1, . . . , n) is defined as follows:

$$
\frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m}\sum_{i=1}^m (h_\theta(x^{(i)})) - y^{(i)})x_j^{(i)}
$$

Note that while this gradient looks identical to the linear regression gradient, the formula is actually different because linear and logistic regression have different definitions of $h_θ(x)$.
Once you are done, we will call your cost_function using the initial
parameters of θ. You should see that the cost is about 0.693.


In [4]:
function cost_function(theta)
    J::Float64 = 0
    grad = zeros(theta)

    h = sigmoid(X * theta)
    J = 1/m * sum(-y .* log.(h) .- (1 .- y) .* log.(1 .- h))
end


function gradient!(grad, theta)
    h = sigmoid(X * theta)
    grad[:] = 1/m * X' * (h .- y)
end



# Setup the data matrix appropriately, and add ones for the intercept term
const m = size(X_data, 1)
const n = size(X_data, 2)

# Add intercept term to X_data and y_data
const X = hcat(ones(m), X_data)
const y = copy(y_data)

# Initialize fitting parameters
initial_theta = zeros(n+1)
grad = zeros(initial_theta)

# Compute and display initial cost and gradient
cost = cost_function(initial_theta)
gradient!(grad, initial_theta)

@show(cost)
@show(grad);

cost = 0.6931471805599452
grad = [-0.1, -12.0092, -11.2628]


### 1.2.3) Learning Parameters using Optim.jl
In this part, you will use the function optimize() from the Optim.jl package to find the optimal parameters of theta.

In the previous assignment, you found the optimal parameters of a linear regression model by implementing gradent descent. You wrote a cost function
and calculated its gradient, then took a gradient descent step accordingly.
This time, instead of taking gradient descent steps, you will use a built-in function called *optimize()*.
It is an optimization solver that finds the minimum of an unconstrained function.
For logistic regression, you want to optimize the cost function J(θ) with parameters θ.
Concretely, you are going to use *optimize()* to find the best parameters θ
for the logistic regression cost function, given a fixed dataset (of X and y
values).

You will pass to *optimize()* the following inputs:

**optimize(cost_function, gradient!, initial_theta, LBFGS())**
where the last argument is the solver to be used to solve the optimization problem.
Feel free to look into the documentation of Optim.jl and check out what other solvers
are implemented.

Once *optimize()* completes, we will call your cost_function
with the optimal parameters of θ.

**You should see that the cost is about 0.203.**

This final θ value will then be used to plot the decision boundary on the
training data, resulting in a figure similar to Figure 2. We also encourage
you to look at the code in *plot_decision_boundary()* to see how to plot such
a boundary using the θ values.

In [5]:
result = optimize(cost_function, gradient!, initial_theta, LBFGS())

theta_opt = result.minimizer
cost_opt = result.minimum

@show(theta_opt)
@show(cost_opt);

theta_opt = [-25.1613, 0.206232, 0.201472]
cost_opt = 0.20349770158943997


In [6]:
function plot_decision_boundary(theta)
    x_plt = [minimum(X[:, 2]) - 2, maximum(X[:, 2]) + 2]
    y_plt = -1/theta[3] .* [theta[1] .+ x_plt .* theta[2]]
    plot_data(X_data, y_data)
    plot!(x_plt, y_plt, color=:blue, label="Decision Boundary",
          xaxis=("Exam-1 score"), yaxis=("Exam-2 score"))
    xlims!(30,100)
    ylims!(30,100)
end

plot_decision_boundary(theta_opt)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 80 
 
 
 90 
 
 
 100 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 80 
 
 
 90 
 
 
 100 
 
 
 Exam-1 score 
 
 
 Exam-2 score 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 admitted 
 
 
 
 
 not admitted 
 
 
 
 Decision Boundary

### 1.2.4) Evaluating Logistic Regression
After learning the parameters, you can use the model to predict whether a
particular student will be admitted. For a student with an Exam 1 score
of 45 and an Exam 2 score of 85, you should expect to see an admission
probability of 0.776. this value is computed by *predict_proba()*

Another way to evaluate the quality of the parameters we have found
is to see how well the learned model predicts on our training set. In this
part, your task is to complete the code in *predict()*. The predict function
will produce “1” or “0” predictions given a dataset and a learned parameter
vector θ.
After you have completed the code in *predict()*, the script will
proceed to report the training accuracy of your classifier by computing the
percentage of examples it got correct.

In [7]:
predict_proba(X, theta) = sigmoid(X * theta)

function predict(X, theta)
    # h = ...
    
    h = predict_proba(X, theta)
    h[h .>= 0.5] = 1
    h[h .< 0.5] = 0
    return round.(Int, h)
end


xtest = [1 45 85]

proba = predict_proba(xtest, theta_opt)
prediction = predict(xtest, theta_opt)

println("Probability of admittance: $proba")

yhat = predict(X, theta_opt)
accuracy = mean(yhat .== y) * 100
println("Model Accuracy: $accuracy %")

Probability of admittance: [0.776291]
Model Accuracy: 89.0 %


### 1.2.5) Manual vs. Automaitc Differentiation
This is an additional section for Julia users only. We quickly examine the effects of manaual vs. automatic differentiation.
It turns out that you can omit the *gradient!()* when calling *optimize()*. In this case a numerical gradient will be automatically computed for you. The up side is that you can be lazy and don't have to analytically compute the gradient yourself. The down side however is a significant increas in computational cost as you will see below.

Use the BenchmarkTools.jl library to examine the effects on the run time when providing gradient functions manually vs. when the built in auto differentiation option is used. By executing the code below you can see that
manually providing the *gradient!()* function improves the run time and decreases memory allocation.  

In [8]:
@benchmark optimize(cost_function, gradient!, initial_theta, BFGS())

BenchmarkTools.Trial: 
  memory estimate:  667.78 KiB
  allocs estimate:  1725
  --------------
  minimum time:     494.540 μs (0.00% GC)
  median time:      516.749 μs (0.00% GC)
  mean time:        561.659 μs (6.69% GC)
  maximum time:     2.456 ms (74.70% GC)
  --------------
  samples:          8759
  evals/sample:     1

In [9]:
@benchmark optimize(cost_function, initial_theta, LBFGS())

BenchmarkTools.Trial: 
  memory estimate:  1.41 MiB
  allocs estimate:  3091
  --------------
  minimum time:     1.667 ms (0.00% GC)
  median time:      1.708 ms (0.00% GC)
  mean time:        1.790 ms (3.68% GC)
  maximum time:     3.743 ms (36.73% GC)
  --------------
  samples:          2779
  evals/sample:     1